In [ ]:
import pandas as pd
import asyncio
import aiohttp
import time
import json

In [ ]:
doktorlar_detay = pd.read_csv('doktorlar_detay.csv')
doktorlar_detay.head()

In [ ]:
data_id = doktorlar_detay['data_id'].dropna().astype(int).to_list()
len(data_id)

In [ ]:


# https://api.doktorsitesi.com/public/question-thread/profile/180089?page=1&limit=100000

# the response is a json object

zero_responses = []
error_data_ids = []

start_time = time.time()
async def fetch(session, url, data_id):
    try:
        async with session.get(url) as response:
            res = await response.json()
            # {"count": "0", "items": []}
            # save the response to a json file with the name of the data_id in the soru_cevap folder
            if res['count'] == '0':
                zero_responses.append(data_id)
                print(f"{url} has no response")
            else:
                with open(f'soru_cevap/{data_id}.json', 'w') as f:
                    json.dump(res, f, ensure_ascii=False)
            
                print(f"{url} is fetched")
    except Exception as e:
        print(f"Error occured while fetching {url}")
        print(e)
        error_data_ids.append(data_id)
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, url in enumerate(urls):
            task = asyncio.create_task(fetch(session, url, data_id[i]))
            tasks.append(task)
        await asyncio.gather(*tasks)
    
urls = [f'https://api.doktorsitesi.com/public/question-thread/profile/{data_id[i]}?page=1&limit=1000000' for i in range(len(data_id))]
await fetch_all(urls)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
len(zero_responses)

In [ ]:
len(error_data_ids)

In [ ]:
# add response count to the doktorlar_detay dataframe and save it to a csv file

for di in data_id:
    try:
        with open(f'soru_cevap/{di}.json') as f:
            res = json.load(f)
            doktorlar_detay.loc[doktorlar_detay['data_id'] == di, 'response_count'] = res['count']
    except:
        doktorlar_detay.loc[doktorlar_detay['data_id'] == di, 'response_count'] = 0

In [ ]:
doktorlar_detay.to_csv('doktorlar_detay.csv', index=False)

In [ ]:
doktorlar_detay.head()

In [ ]:
# get total response count

total_response_count = 0
for di in data_id:
    try:
        with open(f'soru_cevap/{di}.json') as f:
            res = json.load(f)
            total_response_count += int(res['count'])
    except:
        pass

total_response_count

In [ ]:
# convert data_id to int if it is not already int or not nan
doktorlar_detay['data_id'] = doktorlar_detay['data_id'].apply(lambda x: int(x) if not pd.isna(x) else x)

# save it to a csv file
doktorlar_detay.to_csv('doktorlar_detay.csv', index=False)

In [ ]:
doktorlar_detay.head()

## 167.852 responses